## Python Decorators --> Foundations
-------------------------------------

In [16]:
from time import sleep, time

def f():
    sleep(.3)

In [17]:
t = time()
f()
print('f took: ', time() - t)  

f took:  0.30097150802612305


### The 1st simple goal is to run a function inside a function

In [18]:
def measure(func):
    t = time()
    func()
    print(func.__name__, 'took:', time() - t)

In [19]:
measure(f)  

f took: 0.3000626564025879


### Now the extra feature is to provide the inner function the ability to receive multiple arguments or keyword arguments

In [20]:
def f(sleep_time=0.1):
    sleep(sleep_time)

def measure(func, *args, **kwargs):
    t = time()
    func(*args, **kwargs)
    print(func.__name__, 'took:', time() - t)

measure(f, sleep_time=0.3)  # f took: 0.3004162311553955
measure(f, 0.2)  # f took: 0.20028162002563477

f took: 0.3007056713104248
f took: 0.2007913589477539


### Here we add simple housekeeping to ensure that arguments are assign to the right function

In [21]:
def f(sleep_time=0.1):
    sleep(sleep_time)

def measure(func):
    def wrapper(*args, **kwargs):
        t = time()
        func(*args, **kwargs)
        print(func.__name__, 'took:', time() - t)
    return wrapper

f = measure(f)  # decoration point

f(0.2)  # f took: 0.2002875804901123
f(sleep_time=0.3)  # f took: 0.3003721237182617
print(f.__name__)  # wrapper  <- ouch!

f took: 0.2001810073852539
f took: 0.3009052276611328
wrapper


In [23]:
from functools import wraps

def measure(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        t = time()
        func(*args, **kwargs)
        print(func.__name__, 'took:', time() - t)
    return wrapper

@measure
def f(sleep_time=0.1):
    """Initial function! """
    sleep(sleep_time)


f(sleep_time=0.3)  # f took: 0.30039525032043457
print(f.__name__, ':', f.__doc__)
# f : I'm a cat. I love to sleep!

f took: 0.3006265163421631
f : Initial function! 


The f function is decorated by measure, which means that the f function becomes an argument in the inner wrapper function of measure.
However the integrity of the f function is preserved so when called directly it can still be referenced

The ideas si to wrap around the initial function with other functionalities that than to be needed of a recurrent theme, so they can be easily used by any function when needed.

In [27]:
def max_result(threshold):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            result = func(*args, **kwargs)
            if result > threshold:
                print(
                    'Result is too big ({0}). Max allowed is {1}.'
                    .format(result, threshold))
            return result
        return wrapper
    return decorator

@max_result(75)
def cube(n):
    return n ** 3

@max_result(10)
def square(n):
    return n ** 2

@max_result(100)
def multiply(a, b):
    return a * b

In [26]:
print(cube(5))
print(square(4))
print(multiply(a = 10, b=12))
print(multiply(5,5))

Result is too big (125). Max allowed is 75.
125
Result is too big (16). Max allowed is 10.
16
Result is too big (120). Max allowed is 100.
120
25
